# Sistema de Biblioteca: `Libro` y `Usuario`

Este notebook resume **qué hace cada clase**, **por qué los métodos están donde están** y recorre el **flujo paso a paso** con ejemplos ejecutables para practicar antes del parcial.

## Idea de diseño (lo del audio retomado)
- `Libro` sabe **todo sobre sí mismo**: sus datos y si está disponible. Puede **cambiar su propio estado** con `prestar()` / `devolver()`.
- `Usuario` sabe **qué libros tiene ese usuario** y coordina el préstamo/devolución con el libro específico. Por eso `pedir_libro_prestado()` y `devolver_libro_prestado()` viven en `Usuario`: ahí es donde se **registra** qué libros posee cada persona.

Esta separación mantiene la **responsabilidad única** de cada clase y permite escalar (por ejemplo, límites por usuario) sin tocar `Libro`.


## 1) Clase `Libro` (resumen)
- Atributos privados: título, autor, ISBN, editorial, disponibilidad (`True` por defecto).
- Getters/Setters: acceso controlado a los atributos.
- `prestar()` y `devolver()` cambian el estado y **validan** con excepciones si la operación no corresponde.
- `__str__`: representación corta legible.

> Para correr los ejemplos, importamos la clase del archivo `Libro.py`.


In [ ]:
import sys, os
sys.path.append('/mnt/data')  # aseguramos poder importar Libro.py
from Libro import Libro

# Pequeña prueba rápida
libro_demo = Libro('1984', 'Orwell', 'ISBN-1984', 'Debolsillo')
print(libro_demo)                 # usa __str__
print(libro_demo.get_disponible())
libro_demo.prestar()
print(libro_demo.get_disponible())
libro_demo.devolver()
print(libro_demo.get_disponible())

## 2) Clase `Usuario` (explicación)
- Atributos: nombre, DNI, y un **diccionario** `__libros_prestados` cuyas *claves* son ISBN (`str`) y *valores* son objetos `Libro`.
- Al iniciar, ese diccionario empieza **vacío**. Luego se llena cuando el usuario pide libros.
- `pedir_libro_prestado(libro)`: valida que no lo tenga ya, pide al `Libro` que se preste y lo guarda por ISBN.
- `devolver_libro_prestado(libro)`: valida que lo tenga, pide al `Libro` que se devuelva y lo quita del diccionario.
- `ver_libros_prestados()`: devuelve un string legible con todos los libros actuales.


In [ ]:
from typing import List, Dict
from Libro import Libro

class Usuario:
    def __init__(self, nombre: str, dni: str):
        self.__nombre = nombre
        self.__dni = dni
        self.__libros_prestados: Dict[str, Libro] = {}

    def get_nombre(self) -> str:
        return self.__nombre

    def get_dni(self) -> str:
        return self.__dni

    def get_libros_prestados(self) -> List[Libro]:
        return list(self.__libros_prestados.values())

    def set_nombre(self, nombre: str) -> None:
        self.__nombre = nombre

    def pedir_libro_prestado(self, libro: Libro) -> None:
        isbn = libro.get_isbn()
        if isbn in self.__libros_prestados:
            raise ValueError('El usuario ya tiene este libro prestado.')
        try:
            libro.prestar()
            self.__libros_prestados[isbn] = libro
        except ValueError as e:
            raise ValueError(f'No se pudo prestar el libro: {e}')

    def devolver_libro_prestado(self, libro: Libro) -> None:
        isbn = libro.get_isbn()
        if isbn not in self.__libros_prestados:
            raise ValueError('El usuario no tiene este libro prestado.')
        try:
            libro.devolver()
            del self.__libros_prestados[isbn]
        except ValueError as e:
            raise ValueError(f'No se pudo devolver el libro: {e}')

    def ver_libros_prestados(self) -> str:
        if not self.__libros_prestados:
            return 'No tienes libros prestados actualmente'
        libros = f"\nLibros Prestados - {self.__nombre}:\n"
        libros += '=' * 50 + '\n'
        for libro in self.__libros_prestados.values():
            libros += f"{libro}\n"
            libros += '-' * 50 + '\n'
        return libros

    def __str__(self) -> str:
        return f"{self.__nombre} (DNI: {self.__dni})"

## 3) Demo paso a paso
Observá cómo `__libros_prestados` empieza vacío y se va actualizando por ISBN.


In [ ]:
# Creamos libros y un usuario
libro1 = Libro('1984', 'Orwell', 'ISBN-1984', 'Debolsillo')
libro2 = Libro('Fahrenheit 451', 'Bradbury', 'ISBN-0451', 'Minotauro')
ana = Usuario('Ana', '12345678')

print(ana)
print(ana.ver_libros_prestados())  # vacío

# Ana pide el primer libro
ana.pedir_libro_prestado(libro1)
print(ana.ver_libros_prestados())

# Ana pide el segundo libro
ana.pedir_libro_prestado(libro2)
print(ana.ver_libros_prestados())

# Devuelve el primero
ana.devolver_libro_prestado(libro1)
print(ana.ver_libros_prestados())

## 4) Errores controlados (excepciones)
Probá cada caso para ver el `ValueError` correspondiente y entender la validación:
- Prestar dos veces el **mismo** libro al **mismo** usuario.
- Pedir un libro que **ya está prestado** a otra persona.
- Devolver un libro que **no tenés**.


In [ ]:
otro = Usuario('Beto', '22222222')

# Caso 1: mismo usuario intenta prestar 2 veces el mismo libro
try:
    ana.pedir_libro_prestado(libro2)
except ValueError as e:
    print('Caso 1 ->', e)

# Caso 2: libro ya prestado a Ana, ahora lo pide Beto
try:
    otro.pedir_libro_prestado(libro2)
except ValueError as e:
    print('Caso 2 ->', e)

# Caso 3: Beto intenta devolver un libro que no tiene
try:
    otro.devolver_libro_prestado(libro2)
except ValueError as e:
    print('Caso 3 ->', e)